In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import re


In [6]:
index_to_token

{0: 'A',
 1: 'C',
 2: 'D',
 3: 'E',
 4: 'F',
 5: 'G',
 6: 'H',
 7: 'I',
 8: 'K',
 9: 'L',
 10: 'M',
 11: 'N',
 12: 'P',
 13: 'Q',
 14: 'R',
 15: 'S',
 16: 'T',
 17: 'U',
 18: 'V',
 19: 'W',
 20: 'X',
 21: 'Y',
 22: '<OTHER>',
 23: '<START>',
 24: '<END>',
 25: '<PAD>'}

In [5]:
import re

# 아미노산 및 추가적인 토큰 정의
ALL_AAS = 'ACDEFGHIKLMNPQRSTUVWXY'
ADDITIONAL_TOKENS = ['<OTHER>', '<START>', '<END>', '<PAD>']

# 토큰 인덱스 생성
n_aas = len(ALL_AAS)
aa_to_token_index = {aa: i for i, aa in enumerate(ALL_AAS)}
additional_token_to_index = {token: i + n_aas for i, token in enumerate(ADDITIONAL_TOKENS)}
token_to_index = {**aa_to_token_index, **additional_token_to_index}
index_to_token = {index: token for token, index in token_to_index.items()}
n_tokens = len(token_to_index)

def tokenize_seq(seq):
    # OTHER 토큰 인덱스 가져오기
    other_token_index = additional_token_to_index['<OTHER>']
    
    # 정규 표현식 사용하여 특수 변이형과 아미노산 분리
    pattern = r'([A-Z]+)|(\d+)|ins|del|fs|\*|>|_'  # 숫자와 변이형 패턴 포함
    
    tokens = []
    matches = re.finditer(pattern, seq)
    
    for match in matches:
        text = match.group()
        
        # 아미노산 문자일 경우 - 한 글자씩 인식
        if re.match(r'^[A-Z]+$', text) and text not in ["WT", "ins", "del", "fs", "*", ">"]:
            for amino_acid in text:
                tokens.append(aa_to_token_index.get(amino_acid, other_token_index))
        
        # 특수 변이형과 특정 변이형 (WT 등)은 OTHER로 처리
        elif text in ["WT", "ins", "del", "fs", "*", ">"]:
            tokens.append(other_token_index)
    
    # '<START>'와 '<END>'로 감싸서 반환
    return [additional_token_to_index['<START>']] + tokens + [additional_token_to_index['<END>']]

# 예제 테스트
examples = ["WT", "P11_K12insP", "K551del", "R367_A368delinsP", "R330_K370del",
            "R335*", "R335fs", "R551R", "R551K", "447_448RL>RL", 
            "447_448RL>KP", "447_448RL>*P", "447_448RL>P*", "WT"]

# 각 예제에 대해 결과 출력
for example in examples:
    print(f"Input: {example}")
    print(f"Tokenized: {tokenize_seq(example)}\n")


Input: WT
Tokenized: [23, 22, 24]

Input: P11_K12insP
Tokenized: [23, 12, 8, 22, 12, 24]

Input: K551del
Tokenized: [23, 8, 22, 24]

Input: R367_A368delinsP
Tokenized: [23, 14, 0, 22, 22, 12, 24]

Input: R330_K370del
Tokenized: [23, 14, 8, 22, 24]

Input: R335*
Tokenized: [23, 14, 22, 24]

Input: R335fs
Tokenized: [23, 14, 22, 24]

Input: R551R
Tokenized: [23, 14, 14, 24]

Input: R551K
Tokenized: [23, 14, 8, 24]

Input: 447_448RL>RL
Tokenized: [23, 14, 9, 22, 14, 9, 24]

Input: 447_448RL>KP
Tokenized: [23, 14, 9, 22, 8, 12, 24]

Input: 447_448RL>*P
Tokenized: [23, 14, 9, 22, 22, 12, 24]

Input: 447_448RL>P*
Tokenized: [23, 14, 9, 22, 12, 22, 24]

Input: WT
Tokenized: [23, 22, 24]

